In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Matrices

### Representacion

Vamos a estar operando con matrices dispersas con la siguiente representacion en un archivo distribuido donde cada registro es de la forma: (fila,columna,valor).

Por lo tanto este tipo de representacion

In [3]:
data = [
(1,2,4),
(1,5,3),
(2,1,3),
(3,2,2),
(4,4,-1),
(5,1,1),
(5,5,2)]

In [4]:
data

[(1, 2, 4), (1, 5, 3), (2, 1, 3), (3, 2, 2), (4, 4, -1), (5, 1, 1), (5, 5, 2)]

Representa la siguiente matriz dispersa

```
    0  4  0  0  3
    3  0  0  0  0
    0  2  0  0  0
    0  0  0  -1 0
    1  0  0  0  2
```

Notar que **la representacion que hemos hecho esta indexada de 1 en adelante en vez de 0, como se hace en la mayoria de los lenguajes de programacion. Esto es algo a tener en cuenta en las subsiguientes operaciones**

In [5]:
matrixRDD = sc.parallelize(data,8);
matrixRDD.take(20)

[(1, 2, 4), (1, 5, 3), (2, 1, 3), (3, 2, 2), (4, 4, -1), (5, 1, 1), (5, 5, 2)]

### Multiplicacion matriz vector

Buscamos realizar la siguiente operacion:
    
Representa la siguiente matriz dispersa

```
    0  4  0  0  3     1
    3  0  0  0  0     2
    0  2  0  0  0  *  3 
    0  0  0  -1 0     4
    1  0  0  0  2     5
```

Para poder realizar la operacion con las matrices deberia multiplicar cada fila de la matriz por el vector.
En el caso de la matriz dispersa esto es equivalente a **multiplicar cada elemento de la matriz dispersa por el elemento que le corresponde en el vector y acumular por filas.**

Para poder realizar la operacion tenemos que llevar la matriz a una representacion de (fila,columna,valor) en (fila,(columna,valor))
    

In [6]:
#definimos el vector que estaremos multiplicando
vector = [1, 2, 3, 4, 5]

In [7]:
matrixPerRowRDD = matrixRDD.map(lambda x: (x[0],(x[1],x[2])))

In [8]:
matrixPerRowRDD.take(20)

[(1, (2, 4)),
 (1, (5, 3)),
 (2, (1, 3)),
 (3, (2, 2)),
 (4, (4, -1)),
 (5, (1, 1)),
 (5, (5, 2))]

Una vez que la llevamos al formato que necesitamos podemos multiplicar cada elemento de la matriz por el elemento del vector que le corresponde.

**Cada elemento de la matriz nos indica el numero de columna (y ese numero de columna menos 1 nos indica el indice del vector por el que tenemos que multiplicar).**

In [9]:
partialResultRDD = matrixPerRowRDD.map(lambda x: (x[0], vector[x[1][0]-1] * x[1][1]))

In [10]:
partialResultRDD.take(20)

[(1, 8), (1, 15), (2, 3), (3, 4), (4, -4), (5, 1), (5, 10)]

todos los valores parciales por fila que tenemos que agregar fila para obtener el total 

In [11]:
resultPerRow = partialResultRDD.reduceByKey(lambda x,y: x+y)
resultPerRow.take(5)

[(1, 23), (2, 3), (3, 4), (4, -4), (5, 11)]

Notese que el resultado esta expresado como (fila, valor) para llevarlo a una representacion de vector podemos hacer lo siguiente.

In [12]:
result = resultPerRow.map(lambda x: x[1])
result.take(5)

[23, 3, 4, -4, 11]

### Multiplicacion de Matrices

Suponiendo que las dimensiones son compatibles, haremos la multiplicacion de dos matrices dispersas definidas nuevamente como (fila, columna, valor)


```
  1  2  x  5  6  =  19  22
  3  4     7  8     43  50
```

Tenemos que notar que en el caso de las matrices dispersas

In [13]:
# matriz 1
m1 = [(1,1,1),
(1,2,2),
(2,1,3),
(2,2,4)]

# matriz 2
m2 = [(1,1,5),
(1,2,6),
(2,1,7),
(2,2,8)]

In [14]:
m1

[(1, 1, 1), (1, 2, 2), (2, 1, 3), (2, 2, 4)]

In [15]:
m2

[(1, 1, 5), (1, 2, 6), (2, 1, 7), (2, 2, 8)]

In [16]:
m1RDD = sc.parallelize(m1,8)
m2RDD = sc.parallelize(m2,8)

Para realizar el producto entre dos matrices debemos notar que **los elementos de la primer columna de la matriz 1 (1 y 3) siempre se multiplican unicamente por los elementos de la primera fila de la matriz 2 (5 y 6) por lo tanto la estrategia es realizar un join en donde la columna de la matriz 1 coincida con la fila de la matriz 2.**

In [17]:
# llevamos a matriz 1 a una representacion por columna

r1 = m1RDD.map(lambda x: (x[1],(x[0],x[2])))
r1.take(20)

[(1, (1, 1)), (2, (1, 2)), (1, (2, 3)), (2, (2, 4))]

In [18]:
# llevamos a matriz 2 a una representacion por fila

In [19]:
r2 = m2RDD.map(lambda x: (x[0],(x[1],x[2])))
r2.take(20)

[(1, (1, 5)), (1, (2, 6)), (2, (1, 7)), (2, (2, 8))]

luego usando join juntamos los datos que necesitamos procesar en conjunto.

In [20]:
rj = r1.join(r2)
rj.take(20)

[(1, ((1, 1), (1, 5))),
 (1, ((1, 1), (2, 6))),
 (1, ((2, 3), (1, 5))),
 (1, ((2, 3), (2, 6))),
 (2, ((1, 2), (1, 7))),
 (2, ((1, 2), (2, 8))),
 (2, ((2, 4), (1, 7))),
 (2, ((2, 4), (2, 8)))]

Tenemos que multiplicar los valores y el resultado hay que acumularlo en el numero de fila y columna indicado en los registros. Por ejemplo para el registro ``(2, ((2, 4), (1, 7)))`` tenemos que multiplicar ``4 * 7`` y ese resultado acumularlo para la fila 2, columna 1 obteniendo ``((2,1), 28)``.

La idea entonces es acumular por fila y columan para luego acumular usando un reduce para obtener el resultado final

In [21]:
rj2 = rj.map(lambda x:((x[1][0][0], x[1][1][0]), x[1][0][1] * x[1][1][1]))
rj2.take(20)

[((1, 1), 5),
 ((1, 2), 6),
 ((2, 1), 15),
 ((2, 2), 18),
 ((1, 1), 14),
 ((1, 2), 16),
 ((2, 1), 28),
 ((2, 2), 32)]

In [22]:
result = rj2.reduceByKey(lambda x,y: x+y)
result.take(20)

[((2, 2), 50), ((1, 2), 22), ((1, 1), 19), ((2, 1), 43)]

Tener en cuenta que el realizar la operacion los resultados quedan en la representacion **((fila, columna), valor)**. Si quisieramos llevarlo a la representacion original de matriz dispera podriamos hacer la siguiente transformacion

In [23]:
final = result.map(lambda x: (x[0][0], x[0][1], x[1]))
final.take(20)

[(2, 2, 50), (1, 2, 22), (1, 1, 19), (2, 1, 43)]